In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, RidgeClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

%matplotlib inline

/Users/christophermanley/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('./Data/model data 2.csv', index_col=0)

In [3]:
df.head()

,games,goals,goals_against_ev,goals_ev,goals_pp,losses,opp_goals,opp_goals_pp,pdo,pen_kill_pct,...,avg_plus_minus,avg_ops,avg_dps,avg_ps,fenwick_pct,score_balance_pct,ev_goal_diff,special_teams_diff,gf_pg,ga_pg
ind,,,,,,,,,,,,,,,,,,,,,
Nashville Predators_2018,82.0,267.0,145.0,193.0,58.0,18.0,211.0,54.0,101.6,81.94,...,9.185185,1.692593,1.700000,4.103704,49.836000,0.388889,48.0,4.0,3.256098,2.573171
Winnipeg Jets_2018,82.0,277.0,159.0,200.0,64.0,20.0,218.0,50.0,101.0,81.75,...,7.250000,1.792857,1.567857,3.925000,51.357692,0.277778,41.0,14.0,3.378049,2.658537
Tampa Bay Lightning_2018,82.0,296.0,172.0,216.0,66.0,23.0,236.0,64.0,102.0,76.03,...,9.333333,2.103704,1.407407,4.085185,51.576000,0.388889,44.0,2.0,3.609756,2.878049
Boston Bruins_2018,82.0,270.0,161.0,197.0,61.0,20.0,214.0,40.0,100.2,83.67,...,5.419355,1.554839,1.509677,3.512903,53.320690,0.277778,36.0,21.0,3.292683,2.609756
Vegas Golden Knights_2018,82.0,272.0,182.0,218.0,53.0,24.0,228.0,44.0,100.5,81.43,...,5.037037,1.788889,1.518519,3.807407,50.770833,0.277778,36.0,9.0,3.317073,2.780488


### Setting up feature and target variables

In [5]:
X = df.drop(columns=['rank', 'cup_champs', 'team_name'])
y = df[['rank', 'year']]

#### Train, Test Split

- Train, test, split is a little tricky due to the dataset. The purpose of the model is to predict playoff performance based on stats from the regular season. So, I cannot use an automated train, test, split here, as I need training data that contains all of the observations from a given year. Instead I have decided to manually select 8 whole years of data to use as my training data, while holding out 2 whole years to use as my testing data.

In [6]:
X_train = X[(X['year'] != 2018)].drop(columns='year')

X_test = X[X['year'] == 2018].drop(columns='year')

y_test = y[y['year'] == 2018].drop(columns='year')

y_train = y[(y['year'] != 2018)].drop(columns='year')

#### Multiclass Logistic Regression

- Initially I had planned to predict only the Stanley Cup winner. This presented a big problem as there is only 1 cup winner in a given year out of 30 or 31 teams depending on the season. This is a huge class imbalance, coupled with the small number of observations (30/31) in a given year, creating a workable model from that data would be extremely difficult.
- Instead I have chosen to assign a rank to each team in every season for which I have data. The teams are ranked based on where they finished. The Stanley Cup winner is ranked at 1, the runner up at 2, followed by conference final runners-up and so on down to 31. For teams that exited the playoffs in the same round, the teams with the higher point totals in the regular season were ranked higher. This rank value is dropped from me feature set and is the main target variable.
- The Multiclass Logistic Regression will allow me to predict every teams final season ranking based on regular season statistics. In addition to receiving the numerical ranking for each team, I will be able to see the probabilities assigned to those predictions. While predicting how far each team will get in the playoffs is very difficult and very high accuracy is unlikely, assigning probabilities to those predictions is necessary for interpreting results.  

In [11]:
logreg = LogisticRegression(random_state=28, multi_class='multinomial', solver='lbfgs')
model = logreg.fit(X_train, y_train);

/Users/christophermanley/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
model.predict(X_train)
model.score(X_train, y_train)

0.3566666666666667

In [13]:
model.predict_proba(X_train)

array([[8.35183515e-02, 7.93551556e-03, 9.73705831e-02, ...,
        1.69840924e-33, 7.47690573e-34, 1.64303718e-37],
       [8.82690369e-02, 1.12782349e-01, 2.31680075e-01, ...,
        4.19103913e-23, 8.66103599e-24, 3.04594525e-26],
       [8.71271093e-03, 3.70174530e-03, 5.83065076e-02, ...,
        5.05200909e-18, 3.58648232e-18, 3.06353017e-21],
       ...,
       [3.27367622e-22, 6.13527748e-19, 2.27257939e-19, ...,
        1.12698414e-01, 1.70516601e-01, 9.09058975e-02],
       [1.40527931e-18, 1.55733320e-14, 1.26395435e-16, ...,
        1.09573284e-01, 2.93054343e-01, 1.53555346e-01],
       [2.80639963e-20, 2.72002883e-16, 1.67808897e-18, ...,
        1.17513616e-01, 1.91700258e-01, 1.20082043e-01]])

In [14]:
model.predict(X_test)
model.score(X_test, y_test)

0.03225806451612903

##### Initial Analysis:

 - Our model is not very predictive as expected. The model is tasked with predicting 30 different outcomes for 2 years, so 60 in total. The complicated part, however, is that the model has very little data to train on. It is essentially using 240 (8 years x 30 teams) observations to make 60 predictions. Compounding that difficulty is the fact that many of these teams are so tightly packed with very little separating them.
 
 - Lets put together a dataframe of the actual ranks vs. the predicted ranks with the probabilities of the predictions.

In [15]:
predictions = model.predict(X_test)

probs = model.predict_proba(X_test)

In [16]:
ind = pd.Series(y_test.index)
y_t = pd.Series(y_test['rank'])
preds = pd.Series(predictions)

probs_df = pd.DataFrame(probs).round(decimals=3)

In [17]:
results = pd.DataFrame(data=[ind, y_t.values, preds]).T

results.rename(columns={'Unnamed 0': 'Actual Rank', 'Unnamed 1': 'Predicted Rank'}, inplace=True)

probs_max = pd.DataFrame(probs_df.max(axis=1))

In [18]:
results = pd.merge(results, probs_max, left_index=True, right_index=True)

results.rename(columns={0: 'Probability'}, inplace=True)

results

,ind,Actual Rank,Predicted Rank,Probability
0,Nashville Predators_2018,5,9,0.518
1,Winnipeg Jets_2018,3,9,0.277
2,Tampa Bay Lightning_2018,4,10,0.495
3,Boston Bruins_2018,7,6,0.296
4,Vegas Golden Knights_2018,2,7,0.417
5,Washington Capitals_2018,1,7,0.246
6,Toronto Maple Leafs_2018,10,7,0.416
7,Anaheim Ducks_2018,13,8,0.281
8,Minnesota Wild_2018,14,7,0.182
9,Pittsburgh Penguins_2018,8,7,0.368


#### Analysis

- Interestingly enough, when I hold back more years for training and attempt to make predictions on just one year, the mnodel seems to do worse. I'm not sure why this is, but I have decided to change things up a bit.

- Next, I will attempt to create just 6 target classes, 'Cup Winner,' 'Cup Finalist', 'Conference Finalist', '2nd Round', '1st Round', 'Missed Playoffs.' This is an attempt to predict at which round a team will be knocked out of the playoffs.